In [8]:
import warnings
import pandas 
pandas.set_option('mode.use_inf_as_na', True)

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# This is required to catch warnings when the multiprocessing module is used
import os

os.environ["PYTHONWARNINGS"] = "ignore"
# import pertpy as pt
import scanpy as sc

import pertpy as pt
adata = pt.dt.kang_2018()

In [9]:
# adata.var_keys.

In [10]:
# result = []
# json = []
# for row in adata.obs.values:
#     result.append([item for kev, item in enumerate(row) ])
# for row in result: json.append({k: v for k, v in zip(adata.obs.columns.tolist(), row)})

# print(json)
#adata.var.highly_variable.keys

In [11]:
adata.var

,name
index,
AL627309.1,AL627309.1
RP11-206L10.2,RP11-206L10.2
RP11-206L10.9,RP11-206L10.9
FAM87B,FAM87B
LINC00115,LINC00115
...,...
C21orf58,C21orf58
PCNT,PCNT
DIP2A,DIP2A


In [19]:

sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

#hv_genes = filtered_keys = df[df['column_name'] == True].index.tolist()

#normal_genes = filtered_keys = df[df['column_name'] == False].index.tolist()

#print(hv_genes, normal_genes)
hv_genes = (list(adata.var[adata.var['highly_variable'] == True].index))
normal_genes = (list(adata.var_names))

hv_columns = [i for i,val in enumerate(genes) if val in hv_genes]
#hv_genes

In [20]:
len(hv_columns)

659

In [21]:
# highly_variable_columns = []


# for key, gene in enumerate(normal_genes):
#     if gene in hv_genes:
#         highly_variable_columns += key

In [22]:
highly_variable_columns
print(hv_genes, normal_genes)

['HES4', 'ISG15', 'TNFRSF18', 'TNFRSF4', 'NADK', 'CTNNBIP1', 'PGD', 'SRM', 'MRTO4', 'ID3', 'FUCA1', 'CLIC4', 'STMN1', 'FGR', 'IFI6', 'PTAFR', 'SNHG12', 'MARCKSL1', 'C1orf122', 'PRDX1', 'JUN', 'LEPROT', 'TYW3', 'NEXN', 'DNAJB4', 'GBP3', 'GBP1', 'GBP5', 'GCLM', 'C1orf162', 'RHOC', 'TSPAN2', 'PDE4DIP', 'FCGR1A', 'HIST2H2BE', 'MTMR11', 'S100A10', 'S100A11', 'S100A9', 'S100A12', 'S100A8', 'S100A4', 'RAB13', 'RIT1', 'LMNA', 'SEMA4A', 'GPATCH4', 'SH2D2A', 'MNDA', 'AIM2', 'PEA15', 'CD84', 'SLAMF1', 'SLAMF7', 'LY9', 'FCER1G', 'FCGR2A', 'FCGR3A', 'CREG1', 'FASLG', 'RABGAP1L', 'KIAA0040', 'QSOX1', 'GLUL', 'NPL', 'NCF2', 'RGL1', 'C1orf21', 'RGS1', 'RGS2', 'DENND1B', 'IL10', 'G0S2', 'ATF3', 'HLX', 'TARBP1', 'GPR137B', 'KMO', 'OPN3', 'CMPK2', 'RSAD2', 'ID2', 'FAM49A', 'QPCT', 'CYP1B1', 'GALM', 'FOXN2', 'VRK2', 'REL', 'PLEK', 'ANXA4', 'MXD1', 'NAGK', 'TMSB10', 'CAPG', 'VAMP5', 'GNLY', 'CD8A', 'CD8B', 'LINC00152', 'RP11-1399P15.1', 'DUSP2', 'IL1R2', 'LIMS1', 'MIR4435-1HG', 'IL1B', 'IL1RN', 'GYPC', 'HN

In [23]:
#preprocess
#batchcontrol
adata

AnnData object with n_obs × n_vars = 24673 × 15706
    obs: 'nCount_RNA', 'nFeature_RNA', 'tsne1', 'tsne2', 'label', 'cluster', 'cell_type', 'replicate', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.4', 'seurat_clusters'
    var: 'name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    obsm: 'X_pca', 'X_umap'

In [24]:
#clustering
#cell type annotation
#embedding
#reverse transcriptome
#differential expression

#rna velocity -> predicts the future state 
#predict the probability of unseen perturbations
#test: first data set => output list of perturbations 
# take a 2nd dataset and get list of perturbations + their probability 


#representation learning (in particular, self-supervised, multi-view, and transfer learning
#https://registry.opendata.aws/tabula-muris/#usageexamples

#see what preprocessing is needed to get better accuracy

##load data

##import premade model => output list of predicted unseen perturbation

#write custom model. get better accuracy

In [25]:
sc.pp.calculate_qc_metrics(adata)
5

5

In [26]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

def download_parallel(args):
    cpus = cpu_count()
    results = ThreadPool(cpus - 1).imap_unordered(download_url, args)
    for result in results:
        print('url:', result[0], 'time (s):', result[1])

In [27]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [28]:
# gene_expressed = ["string1", "string2", "string3"]

    
# strings = np.array(gene_expressed)
# #adata.obs['genes_expressed'] = strings
# coordinates = []
cx = adata.X
cx = cx.tocoo()

# #delete non highly variable genes from matrix


# for i,j,v in zip(cx.row, cx.col, cx.data):
#     if i in hv_columns: coordinates += [[i,j,v]]
# for coord in coordinates:
#     coord[1] = adata.var_names[coord[1]]

#print(coordinates)

In [39]:
from collections import defaultdict
rowGeneExpression = defaultdict(int)
rowGeneExpression2 = defaultdict(dict)


import math
math.floor
df = adata.obs
for column in hv_columns:
    for row_id in range(math.floor(float(df.shape[0])))[:50]:
        rowGeneExpression[row_id] += adata.X[row_id, column]
        #rowGeneExpression2[row_id][column] = adata.X[row_id, column]


In [40]:
len(rowGeneExpression)

50

In [46]:
df = adata.obs
dependent_variables =  [rowGeneExpression[row] for row in range(df.shape[0])]
df['geneExpressionCount'] = dependent_variables
numerical_values = df.select_dtypes(include=[int, float]).values.tolist()


In [47]:
import torch.optim as optim


criterion = torch.nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [48]:
from torch import tensor

t_dep = tensor(dependent_variables) # pertrubations


In [49]:
t_indep = tensor(numerical_values, dtype=torch.float)
t_indep

tensor([[3017.0000,  877.0000,  -27.6404,  ..., 1704.0000,  711.0000,
          135.0695],
        [2481.0000,  713.0000,  -27.4936,  ..., 1614.0000,  662.0000,
          126.1152],
        [ 703.0000,  337.0000,  -10.4682,  ...,  908.0000,  337.0000,
           26.5247],
        ...,
        [1522.0000,  523.0000,   25.1424,  ..., 1422.0000,  523.0000,
            0.0000],
        [1143.0000,  503.0000,   14.3597,  ..., 1185.0000,  503.0000,
            0.0000],
        [1031.0000,  421.0000,   14.5721,  ..., 1144.0000,  419.0000,
            0.0000]])

In [50]:
t_indep.shape[1]

8

In [51]:

n_coeff = t_indep.shape[1]
coeffs = torch.rand(n_coeff)-0.5
coeffs

tensor([-0.2734,  0.1417,  0.0799,  0.0558, -0.4887,  0.1521, -0.4929, -0.2956])

In [52]:
def acc(coeffs): return (val_dep.bool()==(calc_preds(coeffs, val_indep)>0.1)).float().mean()
def calc_preds(coeffs, indeps): return (coeffs * indeps).sum(axis=1)
def calc_preds(coeffs, indeps): return torch.sigmoid((indeps*coeffs).sum(axis=1))

def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

In [53]:
preds = (t_indep*coeffs).sum(axis=1)
loss = torch.abs(preds-t_dep).mean()
loss = calc_loss(coeffs, t_indep, t_dep)

loss = torch.abs(preds-t_dep).mean()
loss

tensor(421.6961, dtype=torch.float64)

In [54]:
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

In [55]:
vals,indices = t_indep.max(dim=0)
t_indep = t_indep / vals

In [56]:
t_indep*coeffs

tensor([[-0.0698,  0.0451, -0.0503,  0.0194, -0.4398,  0.0860, -0.2471, -0.2415],
        [-0.0574,  0.0366, -0.0500,  0.0375, -0.4398,  0.0815, -0.2301, -0.2255],
        [-0.0163,  0.0173, -0.0190, -0.0078, -0.1466,  0.0458, -0.1171, -0.0474],
        [-0.0791,  0.0437, -0.0443,  0.0265, -0.4398,  0.0877, -0.2270, -0.2335],
        [-0.0731,  0.0571,  0.0508,  0.0314, -0.1955,  0.0937, -0.3226, -0.1932],
        [-0.0432,  0.0326, -0.0009, -0.0330, -0.2443,  0.0770, -0.2204, -0.0581],
        [-0.0264,  0.0224, -0.0289,  0.0261, -0.4398,  0.0584, -0.1516, -0.1388],
        ...,
        [-0.0147,  0.0218, -0.0121, -0.0072, -0.1466,  0.0445, -0.1470, -0.0000],
        [-0.0310,  0.0247,  0.0140,  0.0433, -0.3909,  0.0668, -0.1668, -0.0000],
        [-0.0239,  0.0241,  0.0332,  0.0014, -0.2932,  0.0569, -0.1627, -0.0000],
        [-0.0490,  0.0421, -0.0210,  0.0033, -0.1955,  0.0843, -0.2777, -0.0000],
        [-0.0352,  0.0269,  0.0457,  0.0086, -0.2932,  0.0718, -0.1818, -0.0000],
   

In [57]:
t_indep = t_indep / vals

In [58]:
coeffs.requires_grad_()


tensor([-0.2734,  0.1417,  0.0799,  0.0558, -0.4887,  0.1521, -0.4929, -0.2956], requires_grad=True)

In [59]:
loss = calc_loss(coeffs, t_indep, t_dep)

loss.backward()

In [60]:
coeffs.grad

tensor([     0.0000,      0.0000,      0.0001,      0.0000,      0.0161,      0.0000,      0.0001,     -0.0000])

In [74]:
independent_variables

[[3017.0,
  877.0,
  -27.6403728343912,
  14.9666293340935,
  9.0,
  1704.0,
  711.0,
  135.0694769024849],
 [2481.0,
  713.0,
  -27.4936458839978,
  28.9248851440496,
  9.0,
  1614.0,
  662.0,
  126.11521011590958],
 [703.0,
  337.0,
  -10.4681940980147,
  -5.98438948042782,
  3.0,
  908.0,
  337.0,
  26.524655878543854],
 [3420.0,
  850.0,
  -24.3679974111624,
  20.4292847674974,
  9.0,
  1738.0,
  653.0,
  130.63538390398026],
 [3158.0,
  1111.0,
  27.9521697403372,
  24.1597381598606,
  4.0,
  1857.0,
  928.0,
  108.05580234527588],
 [1869.0,
  635.0,
  -0.470236382288022,
  -25.3987091482051,
  5.0,
  1525.0,
  634.0,
  32.52364665269852],
 [1142.0,
  436.0,
  -15.9061825808314,
  20.0853153562297,
  9.0,
  1157.0,
  436.0,
  77.6361431479454],
 [817.0,
  427.0,
  -6.74039407642886,
  -23.9329848720015,
  5.0,
  961.0,
  427.0,
  27.236886024475098],
 [1582.0,
  558.0,
  -17.682802916336,
  21.0055756282042,
  9.0,
  1397.0,
  557.0,
  99.00012600421906],
 [728.0,
  474.0,
  26.18

In [76]:
independent_variables = pd.DataFrame(numerical_values)





In [77]:
from fastai.data.transforms import RandomSplitter
trn_split,val_split=RandomSplitter(seed=42)(independent_variables)

In [78]:
trn_indep,val_indep = t_indep[trn_split],t_indep[val_split]
trn_dep,val_dep = t_dep[trn_split],t_dep[val_split]
#len(trn_indep),len(val_indep)

In [87]:
loss = calc_loss(coeffs, t_indep, t_dep)
loss.backward()
with torch.no_grad():
    coeffs.sub_(coeffs.grad * 0.1)
    coeffs.grad.zero_()
    print(calc_loss(coeffs, t_indep, t_dep))

tensor(0.5073, dtype=torch.float64)


In [80]:




def update_coeffs(coeffs, lr):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

In [81]:
def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

In [82]:
def init_coeffs(): return (torch.rand(n_coeff)-0.5).requires_grad_()
def init_coeffs(): return (torch.rand(n_coeff, 1)*0.1).requires_grad_()


In [83]:
def train_model(epochs=30, lr=0.01):
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

In [84]:
t_indep.shape

torch.Size([24673, 8])

In [85]:
coeffs = train_model(18, lr=0.2)

RuntimeError: The size of tensor a (19739) must match the size of tensor b (8) at non-singleton dimension 0

In [86]:
coeffs

tensor([-0.2734,  0.1417,  0.0799,  0.0558, -0.4887,  0.1521, -0.4929, -0.2956], requires_grad=True)

In [280]:
indep_cols = ['nCount_RNA' ,                
'nFeature_RNA'    ,             
'tsne1'     ,                 
'tsne2'  ,                   
'cluster'    ,                  
'nCount_SCT'   ,              
'nFeature_SCT',
'geneExpressionCount'
]    
def show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))
show_coeffs()

{'nCount_RNA': tensor(0.3913),
 'nFeature_RNA': tensor(0.0994),
 'tsne1': tensor(0.3359),
 'tsne2': tensor(0.0419),
 'cluster': tensor(-0.1523),
 'nCount_SCT': tensor(-0.4862),
 'nFeature_SCT': tensor(-0.0491),
 'geneExpressionCount': tensor(-0.1332)}

In [281]:
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
def calc_preds(coeffs, indeps): return torch.sigmoid(indeps@coeffs)

calc_preds(coeffs, t_indep)

tensor([0.4953, 0.4954, 0.4983, 0.4955, 0.4997, 0.4979, 0.4959,  ..., 0.4985, 0.4975, 0.4985, 0.4980, 0.4988, 0.4984, 0.4987])

In [119]:
len([prob for prob in calc_preds(coeffs, t_indep) if prob > .5]) # should be 5000

22669

In [120]:

acc(coeffs) 

##
# predictions = tf.nn.sigmoid(predictions)
# predictions = tf.where(predictions < 0.5, 0, 1)

# print('Predictions:\n', predictions.numpy())
# print('Labels:\n', label_batch)

# pass in a h5ad file and get back 
#Predictions:
# [0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0]
# Perturbations:
# [0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0]

tensor(0.0717)

In [121]:
print 50
!pip install --quiet hyperopt
!pip install --quiet "ray[tune]"
!pip install --quiet scvi-colab
from scvi_colab import install

install()


import ray
import scanpy as sc
import scvi
from ray import tune
from scvi import autotune

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (517448773.py, line 1)

In [48]:
def benchmarkResults():
    pass

In [49]:
import scgen
adata.obs.rename({"label": "condition"}, axis=1, inplace=True)
adata.obs["condition"].replace({"ctrl": "control", "stim": "stimulated"}, inplace=True)






# adata_t = adata[
#     ~(
#         (adata.obs["cell_type"] == "CD4 T cells")
#         & (adata.obs["condition"] == "stimulated")
#     )
# ].copy()

# cd4t_stim = adata[
#     (
#         (adata.obs["cell_type"] == "CD4 T cells")
#         & (adata.obs["condition"] == "stimulated")
#     )
# ].copy()








# scgen.SCGEN.setup_anndata(adata_t, batch_key="condition", labels_key="cell_type")


In [50]:
# model = scgen.SCGEN(adata_t, n_hidden=800, n_latent=100, n_layers=2)

In [51]:
# model.train(
#     max_epochs=1, batch_size=32, early_stopping=True, early_stopping_patience=25
# )

In [52]:
# adata_t.obsm["scgen"] = model.get_latent_representation()

In [53]:
# sc.pp.neighbors(adata_t, use_rep="scgen")
# sc.tl.umap(adata)

In [ ]:
# adata_t

In [ ]:
# pred, delta = model.predict(
#     ctrl_key="control", stim_key="stimulated", celltype_to_predict="CD4 T cells"
# )

# # we annotate the predicted cells to distinguish them later from ground truth cells.
# pred.obs["condition"] = "prediction"
# print(pred)

In [ ]:
# pred.obs

In [ ]:

# train = adata
# ctrl_adata = adata[
#     ((adata.obs["cell_type"] == "CD4 T cells") & (adata.obs["condition"] == "control"))
# ]
# stim_adata = train[((train.obs['cell_type'] == 'CD4T') & (train.obs['condition'] == 'stimulated'))]

# # concatenate pred, control and real CD4 T cells in to one object
# eval_adata = ctrl_adata.concatenate(cd4t_stim, pred)

In [ ]:
# sc.tl.pca(eval_adata)
# sc.pl.pca(eval_adata, color="condition", frameon=False)

In [ ]:
# cd4t_adata = adata[adata.obs["cell_type"] == "CD4 T cells"]

In [ ]:
# sc.tl.rank_genes_groups(cd4t_adata, groupby="condition", method="wilcoxon")
# diff_genes = cd4t_adata.uns["rank_genes_groups"]["names"]["stimulated"]
# diff_genes

In [ ]:
# r2_value = model.reg_mean_plot(
#     eval_adata,
#     axis_keys={"x": "predicted stimulated", "y": "stimulated"},
#     gene_list=diff_genes[:10],
#     top_100_genes=diff_genes,
#     labels={"x": "predicted", "y": "ground truth"},
#     show=True,
#     legend=False,
# )

In [ ]:
# sc.pl.violin(eval_adata, keys="ISG15", groupby="condition")

In [ ]:
# import pertpy as pt
# import muon as mu
# import scanpy as sc

In [ ]:
# mdata = pt.dt.papalexi_2021()
# for col in mdata.obs: print(col)

In [ ]:
# sc.pp.normalize_total(mdata["rna"])
# sc.pp.log1p(mdata["rna"])
# sc.pp.highly_variable_genes(mdata["rna"], subset=True)

In [ ]:
# mu.prot.pp.clr(mdata["adt"])

In [ ]:
# sc.pp.pca(mdata["rna"])

In [ ]:
# We calculate neighbors with the cosine distance similarly to the original Seurat implementation
# sc.pp.neighbors(mdata["rna"], metric="cosine")

In [ ]:
# sc.tl.umap(mdata["rna"])

In [ ]:
# sc.pl.umap(mdata["rna"], color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms = pt.tl.Mixscape()

# ms.perturbation_signature(
#     mdata["rna"],
#     pert_key="perturbation",
#     control="NT",
#     split_by="replicate",
#     n_neighbors=20,
# )

In [ ]:
# We create a copy of the object to recalculate the PCA.
# Alternatively we could replace the X of the RNA part of our MuData object with the `X_pert` layer.
# adata_pert = mdata["rna"].copy()
# adata_pert.X = adata_pert.layers["X_pert"]
# sc.pp.pca(adata_pert)
# sc.pp.neighbors(adata_pert, metric="cosine")
# sc.tl.umap(adata_pert)
# sc.pl.umap(adata_pert, color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms.mixscape(adata=mdata["rna"], control="NT", labels="gene_target", layer="X_pert")

In [ ]:
# mdata["rna"].obs

In [ ]:
# pt.pl.ms.perturbscore(
#     adata=mdata["rna"], labels="gene_target", target_gene="IFNGR2", color="orange"
# )

In [ ]:
# sc.settings.set_figure_params(figsize=(10, 10))
# pt.pl.ms.violin(
#     adata=mdata["rna"],
#     target_gene_idents=["NT", "IFNGR2 NP", "IFNGR2 KO"],
#     groupby="mixscape_class",
# )

In [ ]:
# pt.pl.ms.heatmap(
#     adata=mdata["rna"],
#     labels="gene_target",
#     target_gene="IFNGR2",
#     layer="X_pert",
#     control="NT",
# )

In [ ]:
# mdata["adt"].obs["mixscape_class_global"] = mdata["rna"].obs["mixscape_class_global"]
# pt.pl.ms.violin(
#     adata=mdata["adt"],
#     target_gene_idents=["NT", "JAK2", "STAT1", "IFNGR1", "IFNGR2", "IRF1"],
#     keys="PDL1",
#     groupby="gene_target",
#     hue="mixscape_class_global",
# )

In [ ]:
# ms.lda(adata=mdata["rna"], labels="gene_target", layer="X_pert")

In [ ]:
# pt.pl.ms.lda(adata=mdata["rna"])
#https://zenodo.org/record/7058382
# folders = '/home/awahab/llm-testing/data_sets/'
# fp1 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_feature_bc_matrix.h5'
# fp2 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_molecule_info.h5'


# import h5py
# import anndata

# # Read the .h5 File
# def explore_h5py_group(group, indent=0):
#     """Recursively print the contents of an h5py group/dataset."""
#     print(group.data.name)
#     items = sorted(group.items())
#     for name, item in items:
#         if isinstance(item, h5py.Dataset):  # Check if item is a dataset
#             print("  " * indent + f"Dataset: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
#         elif isinstance(item, h5py.Group):  # Check if item is a group
#             print("  " * indent + f"Group: {name}")
#             explore_h5py_group(item, indent + 1)  # Recursive call to explore subgroups

# # Open your HDF5 file
# with h5py.File(folders + fp1, 'r') as f:
#     explore_h5py_group(f)